In [ ]:
# -----------------------------------------------
# Versão do torchtext que foi utilizada é a 0.6.0
# -----------------------------------------------

from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import torchtext
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import random
import time
import re
from random import randint
from googletrans import Translator
import plotly.express as px

In [ ]:
# Configurações Gerais

RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

LEARNING_RATE = 0.005
BATCH_SIZE = 100
NUM_EPOCHS = 30

DEVICE = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 100
HIDDEN_DIM = 256
NUM_CLASSES = 7

In [ ]:
# Verificar se conjunto está ok

df = pd.read_csv('Caminho')
df

In [ ]:
# Algoritmo para parafrasear o conjunto de dados com o modelo pré-treinado gpt-3.
# Aceita somente o inglês, utilizado a base de spam.

openai.api_key='Chave'

new = []
sentence = "Rewrite a phrase preserving their original meaning: " 

for i in range (0,232):
    var = df[df.index == i]
    var = var['Mensagem'].to_string() # Mudar atributo de acordo com o conjunto
    var = re.sub("^\d+\s", "",var).strip()
    
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=sentence+var,
        temperature=0,
        max_tokens=500,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    
    aux = response.choices[0].text[2:]
    new.append(aux)

In [ ]:
# Algoritmo Random Deletion para o conjunto Livraria

new = []

for i in range(0,1163):
    var = df[df.index == i]
    var = var['Titulo'].to_string() 
    var = re.sub("^\d+\s", "",var).strip()
    spl = var.split()
    if len(spl) > 1:
        if len(spl) <= 5:
            rand = randint(0,len(spl)-1)
            spl.pop(rand)
            aux = ' '.join(spl)
            new.append(aux)
        else:
            for n in range(0,4):
                rand = randint(0,len(spl)-1)
                spl.pop(rand)
            aux = ' '.join(spl)
            new.append(aux)
    else:
        new.append(var)

In [ ]:
# Algoritmo Random Deletion para o conjunto de spam

new = []

for i in range(0,232):
    var = df[df.index == i]
    var = var['Mensagem'].to_string() 
    var = re.sub("^\d+\s", "",var).strip()
    spl = var.split()
    if len(spl) > 1:
        if len(spl) <= 5:
            rand = randint(0,len(spl)-1)
            spl.pop(rand)
            aux = ' '.join(spl)
            new.append(aux)
        elif len(spl) <= 35:
            for n in range(0,4):
                rand = randint(0,len(spl)-1)
                spl.pop(rand)
            aux = ' '.join(spl)
            new.append(aux)
        else:
            for n in range(0,10):
                rand = randint(0,len(spl)-1)
                spl.pop(rand)
            aux = ' '.join(spl)
            new.append(aux)
    else:
        new.append(var)

In [ ]:
# Algoritmo back-translation

translator = Translator()

new = []

for i in range(0,232): # tamanho do conjunto
    var = df[df.index == i]
    var = var['Titulo'].to_string() # Mudar atributo de acordo com o conjunto
    var = re.sub("^\d+\s", "",var).strip()
    trans1 = translator.translate(var, dest='pt', src='en').text
    trans2 = translator.translate(trans1, dest='en', src='pt').text
    new.append(trans2)

In [ ]:
# Algoritmo Random Swap

new = []

for i in range(0,1163): # tamanho do conjunto
    var = df[df.index == i]
    var = var['Titulo'].to_string() # Mudar atributo de acordo com o conjunto
    var = re.sub("^\d+\s", "",var).strip()
    spl = var.split()
    if len(spl) > 1:
        for n in range(0,2):
            rand1 = randint(0,len(spl)-1)
            rand2 = randint(0,len(spl)-1)
            aux1 = spl.pop(rand1)
            spl.insert(rand2, aux1)
        aux2 = ' '.join(spl)
        new.append(aux2)
    else:
        new.append(var)

In [ ]:
# Depois de aplicado os algoritmos de data augmentation o mesmo é transformado em um novo dataframe

novo = pd.DataFrame(new)
novo['Genero'] = df['Genero'] # Substituir atributo de acordo com dataset
novo.rename(columns={0: 'Titulo'}, inplace = True) 

fim = df.append(novo, ignore_index=True)

# embaralhar as novas amostras
fim = fim.sample(frac = 1)

fim = fim.reset_index(drop=True)

# Salvar o conjunto com as novas amostras
fim.to_csv('Caminho', index=None)

In [ ]:
# Preparar o conjunto de dados para o torchtext

In [ ]:
# Definindo o processamento dos atributos

TEXT = torchtext.data.Field(
    tokenize='spacy', # Por padrão separa por espaços em branco
    tokenizer_language='pt_core_news_sm' # 'en_core_web_sm' inglês / pt_core_news_sm pt
)

# Definindo o processamento do Rótulo

LABEL = torchtext.data.LabelField(dtype=torch.long)

In [ ]:
# Carregando o conjunto train para utilização no torchtext

fields = [('Titulo', TEXT), ('Genero', LABEL)]

train_data = torchtext.data.TabularDataset(
    path='trainLivraria+BackTranslation.csv', format='csv',
    skip_header=True, fields=fields)

In [ ]:
# Carregando o conjunto test para utilização no torchtext

fields = [('Titulo', TEXT), ('Genero', LABEL)]

test_data = torchtext.data.TabularDataset(
    path='testLivraria+BackTranslation.csv', format='csv',
    skip_header=True, fields=fields)

In [ ]:
# Exemplo de uma linha de como ficou o conjunto

print(vars(test_data.examples[0]))

In [ ]:
# Construindo o vocabulário

TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

print(f'Tamanho vocabulário: {len(TEXT.vocab)}')
print(f'Número de classes: {len(LABEL.vocab)}')

In [ ]:
# palavras mais frequentes e a sua contagem

print(TEXT.vocab.freqs.most_common(20))

In [ ]:
# Verificando os números atribuidos para cada classe

print(LABEL.vocab.stoi)

In [ ]:
# Contagem de amostras por classe

LABEL.vocab.freqs

In [ ]:
# Definindo os Data Loaders

train_loader ,test_loader = \
        torchtext.data.BucketIterator.splits( # Melhores batches com bucketiterator
        (train_data, test_data),
         batch_size=BATCH_SIZE,
         sort_within_batch=True,
         sort_key=lambda x: len(x.Titulo),
         device=DEVICE
    )

In [ ]:
print('Train:')
for batch in train_loader:
    print(f'Tamanho da matriz de texto: {batch.Titulo.size()}')
    print(f'Tamanho do vetor de classe: {batch.Genero.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Tamanho da matriz de texto: {batch.Titulo.size()}')
    print(f'Tamanho do vetor de classe: {batch.Genero.size()}')
    break

In [ ]:


# Construção do Modelo LSTM

class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = torch.nn.LSTM(embedding_dim,
                                 hidden_dim)        
        
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
        

    def forward(self, text):
        #  dimensão do text: [sentence length, batch size]
        
        embedded = self.embedding(text)
        #  dimensão embedded: [sentence length, batch size, embedding dim]
        
        output, (hidden, cell) = self.rnn(embedded)
        #  dimensão output: [sentence length, batch size, hidden dim]
        #  dimensão hidden: [1, batch size, hidden dim]

        hidden.squeeze_(0)
        #  dimensão hidden: [batch size, hidden dim]
        
        output = self.fc(hidden)
        
        return output



In [ ]:
# Inicializando o modelo

torch.manual_seed(RANDOM_SEED)

model = RNN(input_dim=len(TEXT.vocab),
            embedding_dim=EMBEDDING_DIM,
            hidden_dim=HIDDEN_DIM,
            output_dim=NUM_CLASSES
)

model = model.to(DEVICE)

# Utilização do otimizador Adam
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

print(model)

In [ ]:
# Função para computar a acurácia do modelo, tanto para treinamento quanto para teste

def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)
            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
            
    return correct_pred.float()/num_examples * 100

In [ ]:
# Treinamento do conjunto

start_time = time.time()

treinamento = []

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text = batch_data.Titulo.to(DEVICE) # Mudar atributo de acordo com dataset
        labels = batch_data.Genero.to(DEVICE) # Mudar atributo de acordo com dataset
 
        # FORWARD AND BACK PROP
        logits = model(text)
        loss = F.cross_entropy(logits, labels)
        
        optimizer.zero_grad()
        loss.backward()
        
        # Atualizar os parâmetros do modelo
        optimizer.step()
        
        # Imprimindo Época / Tamanho do Batch / Loss
        print (f'Época: {epoch+1}/{NUM_EPOCHS} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        resultado = compute_accuracy(model, train_loader, DEVICE)
        print(f'Acurácia Treinamento: 'f'{resultado:.2f}%')
        treinamento.append(resultado.item())
        
    print(f'Tempo decorrido: {(time.time() - start_time)/60:.2f} min')
    
print(f'Tempo total decorrido: {(time.time() - start_time)/60:.2f} min')

# Depois de treinar o modelo com as repectivas épocas, mostrar a acurácia no conjunto de teste
print(f'Acurácia de Teste: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

In [ ]:
# Plotando gráficamente a acurácia para cada época respectiva

epocas = [range(1, 31)] # Número de épocas utilizadas + 1

fig = px.line(x = epocas, y = treinamento, title='Treinamento Livraria + Back Translation')
fig.update_yaxes(title='Acurácia')
fig.update_xaxes(title='Épocas')
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
# Avaliando o modelo com um novo exemplo

import spacy

nlp = spacy.blank("pt")

def predict(model, sentence):

    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.nn.functional.softmax(model(tensor), dim=1)
    prediction = torch.max(prediction, 1)
    return prediction

predict(model, "Um livro histórico") # Exemplo para a base livraria